In [ ]:
from Web_Page_Class_and_Tests_2.Web_Page_Class import WebPage 

In [ ]:
web_page_example = WebPage(url = "https://www.apple.com")

In [ ]:
web_page_example.view_page()

In [ ]:
web_page_example.path_exists()

In [ ]:
web_page_example.output_dir

In [ ]:
web_page_example.truncated_url + ".html"

In [ ]:
web_page_example.download_page()

In [ ]:
web_page_example.download_page_to_folder()

In [ ]:
web_page_example.view_page()

In [ ]:
web_page_example.output_dir

In [ ]:
import os 

print(os.getcwd())

In [ ]:
web_page_example_2 = WebPage(url = "https://www.apple.com") 

web_page_example_2.download_page_to_folder()

In [ ]:
web_page_example_2.view_page()

In [ ]:
web_page_example_3 = WebPage(url = "https://github.com") 

web_page_example_3.download_page_to_folder()

In [ ]:
web_page_example_3.view_page()

In [ ]:
web_page_example.llm_edit_page()

In [ ]:
web_page_example_4 = WebPage(url = "https://www.selenium.dev")

In [ ]:
web_page_example_4.download_page_to_folder()

In [ ]:
web_page_example_4.view_page()

In [ ]:
web_page_example_4.llm_edit_page()

In [2]:
from bs4 import BeautifulSoup

def split_html(html_content, max_chunk_size=5000):
    soup = BeautifulSoup(html_content, 'html.parser')
    chunks = []
    current_chunk = ''
    current_size = 0

    # Iterate over direct children of the body
    for element in soup.body.contents:
        html_string = str(element)
        element_size = len(html_string)

        if current_size + element_size > max_chunk_size and current_chunk:
            chunks.append(current_chunk)
            current_chunk = ''
            current_size = 0

        current_chunk += html_string
        current_size += element_size

    if current_chunk:
        chunks.append(current_chunk)

    return chunks

In [ ]:
# Read the HTML file
with open('html_links/www.selenium.dev.html', 'r', encoding='utf-8') as file:
    html_content = file.read()

# Split the HTML into chunks
chunks = split_html(html_content)

In [ ]:
print(len(chunks))

In [ ]:
print(chunks[0])

In [ ]:
from langchain_core.prompts import ChatPromptTemplate 
from Web_Page_Class_and_Tests_2.Web_Page_Prompt_Templates.Chunks import template_chunks
from langchain_openai import ChatOpenAI

def llm_edit_chunk(html_chunk: str):
    prompt_template = ChatPromptTemplate.from_template(template = template_chunks) 
    chain = prompt_template | ChatOpenAI(model = "gpt-4o-mini", temperature = 0) 
    output = chain.invoke(input = {"original_html_code_chunk": html_chunk}) 
    return output
    

In [ ]:
example_output = llm_edit_chunk(chunks[0])

In [ ]:
print(example_output.content)

In [ ]:
import webbrowser

with open("example_output_file.html", "w") as file: 
    file.write(example_output.content)

In [ ]:
webbrowser.open(url = "example_output_file.html")

In [ ]:
# Process each chunk with the LLM
import concurrent.futures

# Process each chunk with the LLM concurrently
processed_chunks = []
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(llm_edit_chunk, chunk) for chunk in chunks]
    for future in concurrent.futures.as_completed(futures):
        processed_chunks.append(future.result().content)

# Combine the processed chunks
processed_html_body = ''.join(processed_chunks)

# Reconstruct the HTML with the original head and new body
soup = BeautifulSoup(html_content, 'html.parser')
soup.body.clear()
soup.body.append(BeautifulSoup(processed_html_body, 'html.parser'))

# Write the processed HTML to a new file
with open('processed_www.selenium.dev_full.html', 'w', encoding='utf-8') as file:
    file.write(str(soup))

# Combine the processed chunks
processed_html_body = ''.join(processed_chunks)

# Reconstruct the HTML with the original head and new body
soup = BeautifulSoup(html_content, 'html.parser')
soup.body.clear()
soup.body.append(BeautifulSoup(processed_html_body, 'html.parser'))

# Write the processed HTML to a new file
with open('processed_www.selenium.dev_full_concurrent.html', 'w', encoding='utf-8') as file:
    file.write(str(soup))

In [ ]:
webbrowser.open(url = "processed_www.selenium.dev.html")

In [ ]:
from langchain_openai import ChatOpenAI 
from langchain_core.prompts import ChatPromptTemplate 
from Web_Page_Class_and_Tests_2.Web_Page_Prompt_Templates.Chunks import template_chunks
import asyncio

llm = ChatOpenAI(model = "gpt-4o-mini", temperature = 0) 

template = template_chunks 

prompt = ChatPromptTemplate.from_template(template = template) 

chain = prompt | llm 

In [ ]:
async def run_chain(original_chunk): 
    return await chain.ainvoke({"original_html_code_chunks": original_chunk}) 

async def run_multiple_chains(original_chunks): 
    tasks = [run_chain(chunk) for chunk in original_chunks] 
    outputs = await asyncio.gather(*tasks) 
    return outputs

In [ ]:
example_output_async = asyncio.run(run_multiple_chains(chunks))

In [ ]:
print(example_output_async)

In [ ]:
batch_output = chain.batch(inputs = [chunks])

batch_output

In [3]:
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from bs4 import BeautifulSoup
import asyncio

# Create prompt template for HTML processing
prompt = PromptTemplate(
    input_variables=["html_chunk"],
    template="Summarize this HTML content: {html_chunk}"
)

# Initialize the chain
llm = ChatOpenAI()
chain = LLMChain(llm=llm, prompt=prompt)

# Synchronous batch processing
def process_html_sync(html_content):
    chunks = split_html(html_content)
    # Process all chunks in parallel
    results = chain.batch(
        [{"html_chunk": chunk} for chunk in chunks]
    )
    return results

# Asynchronous batch processing
async def process_html_async(html_content):
    chunks = split_html(html_content)
    # Process all chunks in parallel asynchronously
    results = await chain.abatch(
        [{"html_chunk": chunk} for chunk in chunks]
    )
    return results

# Example usage
html_content = "<html><body><p>Some content</p></body></html>"
# Sync
results = process_html_sync(html_content)

# Async
async def main():
    results = await process_html_async(html_content)

asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop